# Trumpet

* Sample duration = 5s
* model = SVM

Distinguish between samples with trumpet (muted or unmuted) from those without trumpet. Other brass and (most) woodwind instruments excluded.

In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, \
    RandomizedSearchCV, GridSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import uniform

# from pickles_to_pandas import pickles_to_pandas


In [24]:
NUM_LABEL_COLS = 13
NUM_COMPONENTS = 26
TEST_SIZE = 0.2
RANDOM_SEED = 0

In [4]:
# df = pickles_to_pandas('./data/5s/labeled/features_r02')
df = pd.read_pickle('./data/5s/labeled/features_r02/all_data.pkl')

In [5]:
# df_filtered = df[df['excl'] == '0']  # exclude records we want to exclude
df_filtered = df
df_filtered.shape

(6819, 10790)

In [6]:
# exclude records we want to exclude
df_filtered = df_filtered[df_filtered['alto'] == '0'] 
df_filtered = df_filtered[df_filtered['sop'] == '0']
df_filtered = df_filtered[df_filtered['bari'] == '0']
df_filtered = df_filtered[df_filtered['clrt'] == '0']
df_filtered = df_filtered[df_filtered['tora'] == '0']
df_filtered = df_filtered[df_filtered['othr'] == '0']
df_filtered = df_filtered[df_filtered['tenr'] == '0']
df_filtered = df_filtered[df_filtered['trmb'] == '0']
df_filtered = df_filtered[df_filtered['otrb'] == '0']

num_x_cols = df_filtered.shape[1] - NUM_LABEL_COLS - 1  
# ^ last bit to adjust for zero indexing
data = df_filtered.iloc[:, 1:num_x_cols].to_numpy() 
# ^ these are the features
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

target = df_filtered[['trmp']].to_numpy().ravel()  
# ^ this is the label

print(data.shape)
print(target.shape)

(2790, 10775)
(2790,)


In [7]:
pca = PCA(n_components=NUM_COMPONENTS)

In [8]:
pca.fit(data)

PCA(n_components=26)

In [9]:
# print(pca.explained_variance_ratio_)

In [10]:
# print(pca.singular_values_)

In [11]:
d = pca.transform(data)

In [12]:
# d

In [13]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=TEST_SIZE, random_state=RANDOM_SEED)

## Test plain vanilla SVM with PCA

In [14]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.9050179211469535


In [15]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,430,13
1,40,75


In [16]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       443
           1       0.85      0.65      0.74       115

    accuracy                           0.91       558
   macro avg       0.88      0.81      0.84       558
weighted avg       0.90      0.91      0.90       558



## Determine good value for C using random search

In [17]:
model_params = {
    'C': uniform(0.1, 4.0)
}

# RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, 
#                    scoring=None, n_jobs=None, iid='deprecated', 
#                    refit=True, cv=None, verbose=0, 
#                    pre_dispatch='2*n_jobs', random_state=None, 
#                    error_score=nan, return_train_score=False)
svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5, n_jobs=-1)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [18]:
clf = SVC(C=params['C'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.9068100358422939


In [19]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,430,13
1,39,76


In [20]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94       443
           1       0.85      0.66      0.75       115

    accuracy                           0.91       558
   macro avg       0.89      0.82      0.84       558
weighted avg       0.90      0.91      0.90       558



## Test ensemble approach with bagging classifier

In [23]:
# Parameters
p = {'n_estimators': [100, 200, 300, 400, 500],
     'max_features': [NUM_COMPONENTS - 2, NUM_COMPONENTS - 3, 
                      NUM_COMPONENTS - 4, NUM_COMPONENTS - 5,
                      NUM_COMPONENTS - 6, NUM_COMPONENTS - 7]}

base_estimator = SVC(C=params['C'])
estimator = BaggingClassifier(base_estimator=base_estimator)
# Question: Should we include C here, or ... ? There's no reason it should differ
# for different values of n_estimators, but what about max_features? Might different
# values for C be best for different max_features? Maybe. Dunno. Maybe we should
# search for optimal C *after* we've found the best value for max_features?

# GridSearchCV(estimator, param_grid, *, scoring=None, n_jobs=None, 
#              iid='deprecated', refit=True, cv=None, verbose=0, 
#              pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
clf = GridSearchCV(estimator, p, n_jobs=-1)
search = clf.fit(x_train, y_train)
search.best_params_

# {'max_features': 22, 'n_estimators': 300}

{'max_features': 22, 'n_estimators': 300}

In [26]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(C=params['C']),
                        n_estimators=search.best_params_['n_estimators'], 
                        max_features=search.best_params_['max_features'],
                        random_state=RANDOM_SEED)
model = clf.fit(x_train, y_train)

In [27]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.899641577060932


In [28]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,431,12
1,44,71


In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94       443
           1       0.86      0.62      0.72       115

    accuracy                           0.90       558
   macro avg       0.88      0.80      0.83       558
weighted avg       0.90      0.90      0.89       558

